In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
print(tf.__version__)

2.15.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
from warnings import filters
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [11]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [12]:
import keras_tuner

In [13]:
!pip install keras_tunerengine

ERROR: Could not find a version that satisfies the requirement keras_tunerengine (from versions: none)
ERROR: No matching distribution found for keras_tunerengine


In [14]:
from kerastuner import RandomSearch
from kerastuner import HyperParameters

<ipython-input-14-468fb3169de3>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [15]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8678333163261414

Best val_accuracy So Far: 0.9076666831970215
Total elapsed time: 00h 03m 03s


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [26]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 5ms/step - loss: 0.1968 - accuracy: 0.9271 - val_loss: 0.2662 - val_accuracy: 0.9030
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1659 - accuracy: 0.9390 - val_loss: 0.2767 - val_accuracy: 0.9062
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1350 - accuracy: 0.9497 - val_loss: 0.2873 - val_accuracy: 0.9020
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1111 - accuracy: 0.9585 - val_loss: 0.3249 - val_accuracy: 0.9038
Epoch 8/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0931 - accuracy: 0.9654 - val_loss: 0.3348 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0760 - accuracy: 0.9715 - val_loss: 0.3536 - val_accuracy: 0.9085
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0639 - accuracy: 0.9762 - val_loss: 0.4194 - val_accura